# Example of EMM for machine learning reweighting using ADMM optimisation

Using the PIMA diabetes dataset we compare the original dataset with labels to the reweighted dataset with artificial labels. We begin by importing the PIMA diabetes dataset, it is already cleaned and does not have any missing values, outliers, etc. This example is also nice since all features are continuous and the labels are binary.

In [ ]:
# Get raw data, convert to csv, and import
# Update directory
import sys
sys.path.insert(1, '../')
# Import libraries
from emm import *
import os.path
from get_data import get_data
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Data locations
raw_dir = "../assets/data/raw/pima_diabetes/archive.zip"
processed_dir = "../assets/data/processed/pima_diabetes/"

# If processed data does exist convert raw to csv
if not os.path.exists(processed_dir + "diabetes.csv"):
    get_data.RData_to_csv(raw_dir,processed_dir)
    
# Get data into dataframe
df = pd.read_csv(processed_dir + "diabetes.csv")

# Replace 0 with nan
nan_cols = ['Glucose', 'BloodPressure','SkinThickness','Insulin','BMI']
df[nan_cols]=df[nan_cols].replace({'0':np.nan,0:np.nan})

Separating features from labels, useful for training ML models and for re-weighting.

In [ ]:
#train_test_splitting of the dataset
X = df.drop(columns = 'Outcome')
# Getting Predicting Value
y = df['Outcome']

First, we look at the means for each features according to outcome.

In [ ]:
marginals_mean = df.groupby('Outcome').mean()
display(marginals_mean)

We may also want to consider standard deviation of features.

In [ ]:
marginals_std = df.groupby('Outcome').std()
display(marginals_std)

### Matching marginals

First we construct functions $F$ upon which we take expectations of under the weighted measure. Mathematically, we express this as
$$ \mathbb E[F(x)] = \sum^N_{i=1} w_i F(x_i) $$

In [ ]:
features = list(X)
marginals = {}
for feature in features:
    marginals[feature] = ['mean', 'std']
marginals

In [ ]:
loss_0 = []
loss_1 = []
for feature in marginals.keys():
    for fun in marginals[feature]:
        marg = getattr(df[[feature,'Outcome']].groupby('Outcome'),fun)()
        loss_0.append(EqualityLoss(marg.loc[0]))
        loss_1.append(EqualityLoss(marg.loc[1]))

In [ ]:
regularizer = regularizers.EntropyRegularizer(limit=None)
#regularizer = regularizers.ZeroRegularizer()
w_0, out_0, sol_0 = emm(X, marginals, loss_0, regularizer, 
                        optimizer = 'gurobi', verbose=False, rho=25, 
                        eps_abs=1e-6, eps_rel=1e-6)
w_1, out_1, sol_1 = emm(X, marginals, loss_1, regularizer,
                     optimizer='gurobi', verbose=False, rho=25,
                        eps_abs=1e-6, eps_rel=1e-6)

In [ ]:
X_0 = X.copy()
X_0["weights"] = w_0 
X_1 = X.copy()
X_1["weights"] = w_1 

# Set theoretical outcome to train on reweighted datasets
X_0['Outcome'] = 0
X_1['Outcome'] = 1

df_w = pd.concat([X_0,X_1])

In [ ]:
df_valw = df_w.drop(columns=['Outcome','weights']).multiply(df_w['weights'], axis="index")
df_valw['Outcome'] = df_w['Outcome']

In [ ]:
w_marginals = df_valw.groupby('Outcome').sum()

In [ ]:
display(marginals_mean)
display(w_marginals)
display(abs((marginals_mean - w_marginals)))

In [ ]:
std_w = pd.concat([abs(X_0[features] - marginals_mean.loc[0]),abs(X_1[features] - marginals_mean.loc[1])])
std_w = std_w.multiply(df_w['weights'], axis="index")
std_w = pd.concat([std_w, df_w['Outcome']],axis=1)

In [ ]:
display(marginals_std)
display(std_w.groupby('Outcome').sum())
display(abs((marginals_std - std_w.groupby('Outcome').sum())))

In [1]:
import numpy as np
np.random.seed(1)
from emm.losses import *
from emm.regularizers import *
from emm.solvers import *

from emm import emm

n = 20
m = 10000
F = np.random.randn(m, n)
fdes1 = np.random.randn(n // 2)
fdes2 = np.random.randn(n // 2)
losses = [LeastSquaresLoss(fdes1), LeastSquaresLoss(fdes2)]
reg = ZeroRegularizer()

w_admm, out_a = emm(F, None, losses, reg, optimizer="admm", verbose=True)
w_cvx, out_c =  emm(F, None, losses, reg, optimizer="cvx",  verbose=True)

np.testing.assert_allclose(w_admm, w_cvx, atol=1e-3)

Iteration     | ||r||/ε_pri | ||s||/ε_dual
It 000 / 5000 | 7.59251e-01 | 3.79755e+01
It 050 / 5000 | 1.42508e+00 | 1.06213e+02
It 100 / 5000 | 2.21266e+00 | 1.37356e+02
It 150 / 5000 | 2.51468e+00 | 8.90891e+01
It 200 / 5000 | 2.28866e+00 | 8.54704e+01
It 250 / 5000 | 2.08330e+00 | 5.70199e+01
It 300 / 5000 | 1.55383e+00 | 5.29750e+01
It 350 / 5000 | 1.32819e+00 | 4.22324e+01
It 400 / 5000 | 1.05659e+00 | 3.02342e+01
It 450 / 5000 | 8.97482e-01 | 2.73335e+01
It 500 / 5000 | 7.71361e-01 | 2.28772e+01
It 550 / 5000 | 6.29143e-01 | 2.01873e+01
It 600 / 5000 | 5.32145e-01 | 2.05072e+01
It 650 / 5000 | 5.13481e-01 | 1.59781e+01
It 700 / 5000 | 4.56808e-01 | 1.52203e+01
It 750 / 5000 | 4.03740e-01 | 1.50117e+01
It 800 / 5000 | 4.08839e-01 | 1.27206e+01
It 850 / 5000 | 3.05010e-01 | 1.29654e+01
It 900 / 5000 | 3.44122e-01 | 1.02292e+01
It 950 / 5000 | 2.42005e-01 | 1.22503e+01
It 1000 / 5000 | 2.92177e-01 | 9.01658e+00
It 1050 / 5000 | 2.14196e-01 | 1.04338e+01
It 1100 / 5000 | 2.80993e-01 | 

c:\users\branko\documents\git projects\emm\env\lib\site-packages\cvxpy-1.1.13-py3.9-win-amd64.egg\cvxpy\problems\problem.py:1278: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


CVX took 12.72914 seconds


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0.001

Mismatched elements: 17 / 10000 (0.17%)
Max absolute difference: 0.0733305
Max relative difference: 1886.2590502
 x: array([0., 0., 0., ..., 0., 0., 0.])
 y: array([-1.559219e-05,  8.242142e-06, -3.883570e-05, ...,  7.719320e-06,
       -3.882232e-05,  6.432740e-06])